In [1894]:
from datetime import datetime, timedelta
import pandas as pd
from dataclasses import dataclass

@dataclass
class YahooDataSettings:
    ticker:str = 'BTC-GBP'
    interval:str = '1d'
    window_in_days:int = 365
    offset_in_days:int = 0

@dataclass
class LorentzianSettings:
    neighborsCount:int = 8
    maxBarsBack:int = 2000
    useDynamicExits:bool = False
    useEmaFilter:bool = False
    emaPeriod:int = 200
    useSmaFilter:bool = False
    smaPeriod:int = 200
    useKernelSmoothing:bool = False
    lookbackWindow:int = 8
    relativeWeight:float = 8.0
    regressionLevel:int = 25
    crossoverLag:int = 2
    useVolatilityFilter:bool = False
    useRegimeFilter:bool = False
    useAdxFilter:bool = False
    regimeThreshold:float = 0.0
    adxThreshold:int = 0
    use_RSI:bool = True
    RSI_param1:int = 14
    RSI_param2:int = 2
    use_WT:bool = True 
    WT_param1:int = 10
    WT_param2:int = 11
    use_CCI:bool = True
    CCI_param1:int = 20
    CCI_param2:int = 2
    use_ADX:bool = True
    ADX_param1:int = 20
    ADX_param2:int = 2,
    use_MFI:bool = True
    MFI_param1:int = 14

@dataclass
class StrategySettings:
  transcaction_costs:float = .004
  pot_percentage_to_bet:float = 1.0
  initial_cash_percentage:float = 1.0
  initial_value: float = 1000.0

@dataclass
class StrategyReturns:
  initial_value: float
  final_value: float
  portfolio_growth: float
  asset_growth: float
  trade_count: int
  win_percentage: float
  activity_log: str

@dataclass
class LorentzianPredictionResult:
    data_settings: YahooDataSettings
    settings: LorentzianSettings
    returns: StrategyReturns

In [1895]:
import yfinance

def get_data_from_yfinance(settings: YahooDataSettings):
    start_time = datetime.now() - timedelta(days = settings.window_in_days + settings.offset_in_days)
    end_time = datetime.now() - timedelta(days = settings.offset_in_days)

    ticker_data = yfinance.Ticker(settings.ticker)

    df = ticker_data.history(interval=settings.interval, start=start_time, end=end_time)

    new_names = {'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Adj Close': 'adj_close', 'Volume': 'volume'}
    df = df.rename(columns=new_names)
    
    return df

In [1896]:
from advanced_ta import LorentzianClassification
from ta.volume import money_flow_index as MFI

def calculate_predictions(df, settings: LorentzianSettings):
    features = []
    if settings.use_RSI:
        features.append(LorentzianClassification.Feature("RSI", settings.RSI_param1, settings.RSI_param2))
    if settings.use_WT:
        features.append(LorentzianClassification.Feature("WT", settings.WT_param1, settings.WT_param2))
    if settings.use_CCI:
        features.append(LorentzianClassification.Feature("CCI", settings.CCI_param1, settings.CCI_param2))
    if settings.use_ADX:
        features.append(LorentzianClassification.Feature("ADX", settings.ADX_param1, settings.ADX_param2))
    if settings.use_MFI:
        features.append(MFI(df['high'], df['low'], df['close'], df['volume'], settings.MFI_param1))

    # df here is the dataframe containing stock data as [['open', 'high', 'low', 'close', 'volume']]
    lc = LorentzianClassification(
        df,    
        features,
        settings=LorentzianClassification.Settings(
            source='close',
            neighborsCount=settings.neighborsCount,
            maxBarsBack=settings.maxBarsBack,
            useDynamicExits=settings.useDynamicExits
        ),
        filterSettings=LorentzianClassification.FilterSettings(
            useVolatilityFilter=settings.useVolatilityFilter,
            useRegimeFilter=settings.useRegimeFilter,
            useAdxFilter=settings.useAdxFilter,
            regimeThreshold=settings.regimeThreshold,
            adxThreshold=settings.adxThreshold,
            kernelFilter = LorentzianClassification.KernelFilter(
                useKernelSmoothing = settings.useKernelSmoothing,
                lookbackWindow = settings.lookbackWindow,
                relativeWeight = settings.relativeWeight,
                regressionLevel = settings.regressionLevel,
                crossoverLag = settings.crossoverLag
            )
        )
    )

    return lc

In [1897]:
def calculate_performance(lc, settings: StrategySettings) -> StrategyReturns: 
  first_price = lc.data.iloc[0]['open']
  last_price = lc.data.tail(1).iloc[0]['open']
  max_price = lc.data['high'].max()
  min_price = lc.data['low'].min()
  price_range = max_price - min_price

  transcaction_costs = settings.transcaction_costs
  pot_percentage_to_bet = settings.pot_percentage_to_bet
  initial_cash_percentage = settings.initial_cash_percentage

  initial_value = settings.initial_value * 100
  initial_cash = (initial_value * initial_cash_percentage)
  initial_shares = ((initial_value * (1- initial_cash_percentage)) * (1 - transcaction_costs)) / first_price
  
  activity_log = f'Initial cash is £{initial_cash / 100} and initial shares are {initial_shares}'
  activity_log +='\n---------------------\n'

  cash = initial_cash
  shares = initial_shares

  is_long = True
  trade_size = 0
  trade_value = 0
  trade_count = 0
  win_count = 0

  for index, row in lc.data.iterrows():
    price = row['close']

    if not pd.isna(row['endLongTrade']) and trade_size > 0 and is_long:
      shares -= trade_size
      cash_value = trade_size * price * (1 - transcaction_costs)
      cash += cash_value
      trade_count += 1
      if (cash_value > trade_value):
        win_count += 1

      activity_log +=f'Closed long trade for {trade_size} shares at {price}. Cashed £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}'

      trade_size = 0
      trade_value = 0
    elif not pd.isna(row['endShortTrade']) and trade_size > 0 and not is_long:
      shares += trade_size
      cash_value = trade_size * price * (1 + transcaction_costs)
      cash -= cash_value
      trade_count += 1
      if (cash_value < trade_value):
        win_count += 1

      activity_log +=f'Closed short trade for {shortTradeSize} shares at {price}. Paid £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}'

      trade_size = 0
      trade_value = 0
    elif not pd.isna(row['startLongTrade']) and trade_size == 0:
      cash_value = cash * pot_percentage_to_bet
      cash -= cash_value
      trade_size = (cash_value * (1 - transcaction_costs)) / price
      shares += trade_size
      is_long = True
      trade_value = cash_value

      activity_log +=f'Opened long trade for {trade_size} shares at {price}. Paid £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}'
    elif not pd.isna(row['startShortTrade']) and trade_size == 0:
      shortTradeSize = shares * pot_percentage_to_bet
      shares -= shortTradeSize
      cash_value = shortTradeSize * price * (1 - transcaction_costs)
      cash += cash_value
      is_long = False
      trade_value = cash_value

      activity_log +=f'Opened short trade for {shortTradeSize} shares at {price}. Cashed £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}'
    else:
      continue

  value = cash +  (shares * last_price) * (1 - transcaction_costs)
  initial_value_gbp = initial_value / 100
  final_value_gbp = value / 100
  portfolio_growth_perc = ((value - initial_value) / initial_value) * 100
  asset_growth_perc = ((last_price - first_price) / first_price) * 100
  if trade_count > 0:
    win_percentage = (win_count / trade_count) * 100
  else:
    win_percentage = 0

  activity_log +='\n---------------------\n'
  activity_log +=f'Initial value: {initial_value_gbp}'
  activity_log +=f'Final value: {final_value_gbp}'
  activity_log +=f'Number of trades: {trade_count}'
  activity_log +=f'Win percentage: {win_percentage} %'
  activity_log +=f'Portfolio growth: {portfolio_growth_perc} %'
  activity_log +=f'Asset growth: {asset_growth_perc} %'

  return StrategyReturns(
    initial_value_gbp,
    final_value_gbp,
    portfolio_growth_perc,
    asset_growth_perc,
    trade_count,
    win_percentage,
    activity_log,
  )

In [1898]:
import copy

def run_lorentzian_strategy_test(data_settings: YahooDataSettings, prediction_settings: LorentzianSettings, strategy_settings: StrategySettings) -> LorentzianPredictionResult:
    df = get_data_from_yfinance(data_settings)
    lc = calculate_predictions(df, prediction_settings)
    returns =  calculate_performance(lc, strategy_settings)
    
    return LorentzianPredictionResult(copy.deepcopy(data_settings), copy.deepcopy(prediction_settings), returns)

In [1935]:
period_in_days = 365
periods = 10

data_settings = YahooDataSettings(
    ticker = 'BTC-GBP', interval = '1d', window_in_days = period_in_days, offset_in_days = 0
)

prediction_settings = LorentzianSettings(
    neighborsCount = 8, maxBarsBack = 2000, useDynamicExits = True,
    useEmaFilter = False, emaPeriod = 200,
    useSmaFilter = False, smaPeriod = 200,
    useKernelSmoothing = True, lookbackWindow = 12, relativeWeight = 8.0, regressionLevel = 30, crossoverLag = 0,
    useVolatilityFilter = False,
    useRegimeFilter = True, regimeThreshold = -1,
    useAdxFilter = False, adxThreshold = 20,
    use_RSI = False, RSI_param1 = 28, RSI_param2 = 7,
    use_WT = True, WT_param1 = 5, WT_param2 = 11,
    use_CCI = True, CCI_param1 = 2, CCI_param2 = 2,
    use_ADX = True, ADX_param1 = 20, ADX_param2 = 2,
    use_MFI = False, MFI_param1 = 14
)

strategy_settings = StrategySettings(
  transcaction_costs = .004, pot_percentage_to_bet = 1.0, initial_cash_percentage = 1, initial_value = 1000.0
)

returns = []

# for i in range(10):
#   prediction_settings.WT_param1  += 1 
for j in range(periods):
  returns.append(run_lorentzian_strategy_test(data_settings, prediction_settings, strategy_settings))
  data_settings.offset_in_days += period_in_days
    
dicts = [{**vars(r.data_settings), **vars(r.returns), **vars(r.settings)} for r in returns]
perf = pd.DataFrame.from_dict(dicts)

wins_average = perf['win_percentage'].mean()

returns_average = perf['portfolio_growth'].mean()
returns_min = perf['portfolio_growth'].min()
returns_max = perf['portfolio_growth'].max()

asset_average =  perf['asset_growth'].mean()
asset_min =  perf['asset_growth'].min()
asset_max =  perf['asset_growth'].max()

print(f'Wins: {wins_average} %')
print('\n----------------\n')
print(f'Average return: {returns_average} %')
print(f'Average asset return: {asset_average} %')
print('\n----------------\n')
print(f'Worst period: {returns_min} %')
print(f'Asset worst period: {asset_min} %')
print('\n----------------\n')
print(f'Best period: {returns_max} %')
print(f'Asset best period: {asset_max} %')

Wins: 61.57142857142857 %

----------------

Average return: 30.797221026258846 %
Average asset return: 111.60931479285333 %

----------------

Worst period: 0.0 %
Asset worst period: -48.4256445063246 %

----------------

Best period: 135.9170850839152 %
Asset best period: 422.0471427964637 %


In [1936]:
page = 0
items_per_page = 10

perf[page * items_per_page: (page * items_per_page) + items_per_page]

,ticker,interval,window_in_days,offset_in_days,initial_value,final_value,portfolio_growth,asset_growth,trade_count,win_percentage,...,WT_param1,WT_param2,use_CCI,CCI_param1,CCI_param2,use_ADX,ADX_param1,ADX_param2,use_MFI,MFI_param1
0,BTC-GBP,1d,365,0,1000.0,1220.865356,22.086536,146.972436,7,57.142857,...,5,11,True,2,2,True,20,2,False,14
1,BTC-GBP,1d,365,365,1000.0,1275.657969,27.565797,-8.973847,6,50.000000,...,5,11,True,2,2,True,20,2,False,14
2,BTC-GBP,1d,365,730,1000.0,1165.022084,16.502208,-8.270407,3,100.000000,...,5,11,True,2,2,True,20,2,False,14
3,BTC-GBP,1d,365,1095,1000.0,2359.170851,135.917085,235.109095,5,80.000000,...,5,11,True,2,2,True,20,2,False,14
4,BTC-GBP,1d,365,1460,1000.0,1032.937377,3.293738,26.101293,2,100.000000,...,5,11,True,2,2,True,20,2,False,14
5,BTC-GBP,1d,365,1825,1000.0,1570.847428,57.084743,8.328253,7,71.428571,...,5,11,True,2,2,True,20,2,False,14
6,BTC-GBP,1d,365,2190,1000.0,1057.608244,5.760824,172.812022,3,66.666667,...,5,11,True,2,2,True,20,2,False,14
7,BTC-GBP,1d,365,2555,1000.0,1221.471504,22.147150,422.047143,7,57.142857,...,5,11,True,2,2,True,20,2,False,14
8,BTC-GBP,1d,365,2920,1000.0,1176.141290,17.614129,170.392805,6,33.333333,...,5,11,True,2,2,True,20,2,False,14
9,BTC-GBP,1d,365,3285,1000.0,1000.000000,0.000000,-48.425645,0,0.000000,...,5,11,True,2,2,True,20,2,False,14


In [1901]:
#data_folder = '../data/lorentzian'

#lc.dump(f'{data_folder}/{ticker}_result.csv')
#lc.plot(f'{data_folder}/{ticker}_result.jpg')